In [ ]:
import os
os.environ['JAVA_HOME'] = '/home/aiops/zhuty/jdk-11.0.20'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']


In [ ]:
from pyserini.search.lucene import LuceneSearcher
from pyserini.index.lucene import IndexReader

index_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_20b_id_added/bm25_index"
index_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/bm25_index"
index_reader = IndexReader(index_path)
index_reader.stats()

In [ ]:
doc_vector = index_reader.get_document_vector('0_0')
print(doc_vector)

In [ ]:
searcher = LuceneSearcher(index_path)
doc = searcher.doc('0_0')
print(doc.raw())
print(doc.contents())

In [ ]:
doc.contents()

In [ ]:
doc.lucene_document(), doc.raw()


In [ ]:
hits = searcher.search('What is the best way to prepare for a job interview?', k=100)
for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

In [ ]:
import os

In [ ]:
import json
docid = "3_10026"
chunk_id, seq_id = docid.split("_")
base_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/train"
base_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/queries"
jsonl_file = os.path.join(base_path, "chunk_{}.jsonl".format(chunk_id))
with open(jsonl_file, "r") as f:
    for line in f:
        line = line.strip()
        if line:
            data = json.loads(line)
            if data["id"] == docid:
                # print(data)
                print(len(data['title'].split()))
                break

In [ ]:
data

In [ ]:
for chunk_num in range(0, 90):
    print(f'sailctl job create bm25search{chunk_num}  --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing; bash search_bm25.sh {chunk_num} " --debug')

In [ ]:
sailctl job create bm25search0  --args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing; bash search_bm25.sh 0 " --debug

Understand how tinyllama works

In [10]:
import json
import os

lengths = []

for i in range(0, 512):
    # read jsonl file
    file_path = f"/home/aiops/zhuty/ret_pretraining_data/c4_news/train/chunk_{i}.jsonl"

    lines = []
    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                data = json.loads(line)
                lines.append(data)
    lengths.append(len(lines))


In [11]:
lengths

[26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,
 26953,
 26953,
 26952,
 26953,
 26953,


In [12]:
sum(lengths)

13799838

In [13]:
13799838/512

26952.80859375

In [2]:
import os


In [6]:
path = "/home/aiops/zhuty/ret_pretraining_data/id_added/redpajama_2b/generated_queries"
for i in range(0,89):
    if not os.path.exists(os.path.join(path, f"_chunk_{i}_results.json")):
        print(i)

4
9
16
17
18
19
23
27
28
29
31
32
33
35
37
38
39
46
47
48
49
50
55
57
58
59
64
65
67
68
69
70
73
76
77
79
83
84
88


In [ ]:
step = 50
commands = []
for i in range(0, 1200, step):
   if i==0:
        continue
# missing = ['chunk_234', 'chunk_254', 'chunk_301', 'chunk_304', 'chunk_303', 'chunk_302', 'chunk_300', 'chunk_305', 'chunk_510', 'chunk_290', 'chunk_235', 'chunk_511']
# for i in missing:
#     i = i.split("_")[1]
    commands.append((f"""sleep 5; sailctl job create bm25search{i}  --debug -f ~/Downloads/cpu_only_values.yaml  --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/ ; bash range_search_bm25.sh c4_news {i} {i}  " ;"""))
print(" ".join(commands))

In [1]:
dataset_name = "c4_news_adv_8k"

In [5]:
if "2b_tokens" in dataset_name or "2b" in dataset_name:
    max_step = 10000
elif "20b_tokens" in dataset_name or "20b" in dataset_name:
    max_step = 40000
elif "c4_news" in dataset_name or "wiki" in dataset_name:
    # around 9b tokens?
    max_step = 25000
else:
    raise ValueError("Invalid dataset name")

ValueError: Invalid dataset name

In [3]:
max_step

10000